In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cassandra-udyam-2022/sampleSubmission.csv
/kaggle/input/cassandra-udyam-2022/trainData.csv
/kaggle/input/cassandra-udyam-2022/testData.csv


In [2]:
train_df = pd.read_csv(r'../input/cassandra-udyam-2022/trainData.csv')
test_df = pd.read_csv(r"../input/cassandra-udyam-2022/testData.csv")

In [3]:
train_df['Invoice_Date'] = pd.to_datetime(train_df['Invoice_Date']) 
train_df['Due_Date'] = pd.to_datetime(train_df['Due_Date'])
train_df['Created'] = pd.to_datetime(train_df['Created'])

test_df['Invoice_Date'] = pd.to_datetime(test_df['Invoice_Date']) 
test_df['Due_Date'] = pd.to_datetime(test_df['Due_Date'])
test_df['Created'] = pd.to_datetime(test_df['Created'])

In [4]:
train_df.drop('Created',axis=1,inplace=True)
test_df.drop('Created',axis=1,inplace=True)

In [5]:
train_df['Due_days'] = (train_df['Due_Date'] - train_df['Invoice_Date']).dt.days.astype('int64')
test_df['Due_days'] = (test_df['Due_Date'] - test_df['Invoice_Date']).dt.days.astype('int64')

In [6]:
train_df['inv_year'] = train_df['Invoice_Date'].dt.year.astype('int64')
train_df['inv_month'] = train_df['Invoice_Date'].dt.month.astype('int64')
train_df['inv_day'] = train_df['Invoice_Date'].dt.day.astype('int64')
train_df['inv_day_of_week'] = train_df['Invoice_Date'].dt.day_of_week.astype('int64')
train_df['inv_weeknd'] = (train_df['inv_day_of_week'] > 4).astype(int)

test_df['inv_year'] = test_df['Invoice_Date'].dt.year.astype('int64')
test_df['inv_month'] = test_df['Invoice_Date'].dt.month.astype('int64')
test_df['inv_day'] = test_df['Invoice_Date'].dt.day.astype('int64')
test_df['inv_day_of_week'] = test_df['Invoice_Date'].dt.day_of_week.astype('int64')
test_df['inv_weeknd'] = (test_df['inv_day_of_week'] > 4).astype(int)

In [7]:
train_df['due_year'] = train_df['Due_Date'].dt.year.astype('int64')
train_df['due_month'] = train_df['Due_Date'].dt.month.astype('int64')
train_df['due_day'] = train_df['Due_Date'].dt.day.astype('int64')
train_df['due_day_of_week'] = train_df['Due_Date'].dt.day_of_week.astype('int64')
train_df['due_weeknd'] = (train_df['due_day_of_week'] > 4).astype(int)

test_df['due_year'] = test_df['Due_Date'].dt.year.astype('int64')
test_df['due_month'] = test_df['Due_Date'].dt.month.astype('int64')
test_df['due_day'] = test_df['Due_Date'].dt.day.astype('int64')
test_df['due_day_of_week'] = test_df['Due_Date'].dt.day_of_week.astype('int64')
test_df['due_weeknd'] = (test_df['due_day_of_week'] > 4).astype(int)

In [8]:
train_df.drop(['Invoice_Date','Due_Date','Outstanding'],axis=1,inplace=True)
test_df.drop(['Invoice_Date','Due_Date','Outstanding'],axis=1,inplace=True)

In [9]:
temp = pd.concat([train_df.drop('Number_of_Days_until_Payment',axis=1),test_df],ignore_index=True,axis=0) 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
temp['Vendor_Name'] = le.fit_transform(temp['Vendor_Name'])
temp

,Description,Vendor_Name,Amount,Settled,Due_days,inv_year,inv_month,inv_day,inv_day_of_week,inv_weeknd,due_year,due_month,due_day,due_day_of_week,due_weeknd
0,Milk x 7 ltrs,789,672.78,672.78,30,2011,4,26,1,0,2011,5,26,3,0
1,Office Stationery,2920,5101.98,5101.98,30,2011,5,24,1,0,2011,6,23,3,0
2,Milk x 10 ltrs,2058,7422.78,7422.78,30,2011,3,24,3,0,2011,4,23,5,1
3,Annual Fee,1842,11.98,11.98,14,2011,5,10,1,0,2011,5,24,1,0
4,NaN,3011,5501.98,5501.98,1,2011,9,5,0,0,2011,9,6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9873,NaN,3266,4201.98,4201.98,14,2016,8,29,0,0,2016,9,12,0,0
9874,NaN,3267,93.18,93.18,7,2014,7,28,0,0,2014,8,4,0,0
9875,NaN,3270,1255.98,1255.98,4,2017,9,2,5,1,2017,9,6,2,0
9876,NaN,3272,678.78,58.38,30,2015,1,2,4,0,2015,2,1,6,1


In [10]:
train_df1 = temp.head(train_df.shape[0])
train_df1

,Description,Vendor_Name,Amount,Settled,Due_days,inv_year,inv_month,inv_day,inv_day_of_week,inv_weeknd,due_year,due_month,due_day,due_day_of_week,due_weeknd
0,Milk x 7 ltrs,789,672.78,672.78,30,2011,4,26,1,0,2011,5,26,3,0
1,Office Stationery,2920,5101.98,5101.98,30,2011,5,24,1,0,2011,6,23,3,0
2,Milk x 10 ltrs,2058,7422.78,7422.78,30,2011,3,24,3,0,2011,4,23,5,1
3,Annual Fee,1842,11.98,11.98,14,2011,5,10,1,0,2011,5,24,1,0
4,NaN,3011,5501.98,5501.98,1,2011,9,5,0,0,2011,9,6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8748,NaN,131,551.98,551.98,30,2017,9,29,4,0,2017,10,29,6,1
8749,NaN,919,386.98,386.98,30,2017,9,29,4,0,2017,10,29,6,1
8750,NaN,176,279.81,279.81,1,2017,10,31,1,0,2017,11,1,2,0
8751,NaN,200,145.98,145.98,14,2017,10,23,0,0,2017,11,6,0,0


In [11]:
test_df1 = temp.tail(test_df.shape[0])
test_df1

,Description,Vendor_Name,Amount,Settled,Due_days,inv_year,inv_month,inv_day,inv_day_of_week,inv_weeknd,due_year,due_month,due_day,due_day_of_week,due_weeknd
8753,Annual Fee,2,11.98,11.98,30,2015,11,30,0,0,2015,12,30,2,0
8754,NaN,6,56.98,56.98,30,2016,9,6,1,0,2016,10,6,3,0
8755,NaN,8,61.98,61.98,14,2016,2,29,0,0,2016,3,14,0,0
8756,NaN,10,674.30,674.30,14,2013,8,23,4,0,2013,9,6,4,0
8757,NaN,12,302.94,302.94,30,2017,5,10,2,0,2017,6,9,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9873,NaN,3266,4201.98,4201.98,14,2016,8,29,0,0,2016,9,12,0,0
9874,NaN,3267,93.18,93.18,7,2014,7,28,0,0,2014,8,4,0,0
9875,NaN,3270,1255.98,1255.98,4,2017,9,2,5,1,2017,9,6,2,0
9876,NaN,3272,678.78,58.38,30,2015,1,2,4,0,2015,2,1,6,1


In [12]:
train_df1.drop(['Description'],axis=1,inplace=True)
test_df1.drop(['Description'],axis=1,inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
fq = temp.groupby('Vendor_Name').size()/len(train_df1)   

train_df1.loc[:, "{}_freq_encode".format('Vendor_Name')] = train_df1['Vendor_Name'].map(fq)  
test_df1.loc[:, "{}_freq_encode".format('Vendor_Name')] = test_df1['Vendor_Name'].map(fq)  

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [14]:
cols = ['Amount',
         'Settled',
         'Due_days',
         'inv_year',
         'inv_month',
         'inv_day',
         'inv_day_of_week',
         'inv_weeknd',
         'due_year',
         'due_month',
         'due_day',
         'due_day_of_week',
         'due_weeknd','Vendor_Name_freq_encode']

In [15]:
from sklearn.preprocessing import QuantileTransformer

for col in cols:
  quantile = QuantileTransformer(output_distribution='normal')
  train_df[col] = quantile.fit_transform(train_df1[col].values.reshape(-1, 1))
  test_df[col] = quantile.transform(test_df1[col].values.reshape(-1, 1))  

In [16]:
train_df


,Description,Vendor_Name,Amount,Settled,Number_of_Days_until_Payment,Due_days,inv_year,inv_month,inv_day,inv_day_of_week,inv_weeknd,due_year,due_month,due_day,due_day_of_week,due_weeknd,Vendor_Name_freq_encode
0,Milk x 7 ltrs,David Taylor,0.720298,0.758085,13,0.341493,-5.199338,-0.568686,0.814401,-0.524976,-5.199338,-5.199338,-0.365531,0.746351,-0.180377,-5.199338,-0.611996
1,Office Stationery,Stephen Wright MD,1.659790,1.739204,38,0.341493,-5.199338,-0.312376,0.613509,-0.524976,-5.199338,-5.199338,-0.123259,0.472407,-0.180377,-5.199338,-5.199338
2,Milk x 10 ltrs,Mark Cordova,1.913885,1.965348,61,0.341493,-5.199338,-0.868016,0.613509,0.432104,-5.199338,-5.199338,-0.624145,0.472407,0.893908,5.199338,-5.199338
3,Annual Fee,Kimberly White,-2.258898,-2.043115,62,-0.866189,-5.199338,-0.312376,-0.496390,-0.524976,-5.199338,-5.199338,-0.365531,0.558392,-1.157061,-5.199338,-0.611996
4,NaN,Teresa Marshall,1.712134,1.795634,2,-5.199338,-5.199338,0.633317,-0.961429,-5.199338,-5.199338,-5.199338,0.592444,-0.933910,-1.157061,-5.199338,-0.473810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8748,NaN,Amy Sawyer,0.627608,0.668556,34,0.341493,5.199338,0.633317,1.218002,0.983573,-5.199338,5.199338,0.873515,1.109378,5.199338,5.199338,-5.199338
8749,NaN,Edward Santiago,0.442799,0.488904,34,0.341493,5.199338,0.633317,1.218002,0.983573,-5.199338,5.199338,0.873515,1.109378,5.199338,5.199338,-5.199338
8750,NaN,Angela Becker,0.232633,0.276702,5,-5.199338,5.199338,0.908975,5.199338,-0.524976,-5.199338,5.199338,1.202347,-5.199338,-0.676854,-5.199338,-5.199338
8751,NaN,Anna Carter,-0.202098,-0.160010,10,-0.866189,5.199338,0.908975,0.523537,-5.199338,-5.199338,5.199338,1.202347,-0.933910,-5.199338,-5.199338,-5.199338


In [17]:
X = train_df1.values
y = train_df['Number_of_Days_until_Payment']

test = test_df1

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2084)

In [19]:
xgb_params = {'learning_rate': 0.21332121304895782,
 'reg_lambda': 0.015832777652673033,
 'reg_alpha': 4.004765999790442e-06,
 'subsample': 0.7633583263826937,
 'colsample_bytree': 0.8016447134543976,
 'max_depth': 7
             ,'n_estimators':15,'eval_metric':'rmse'}

In [20]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

xgbr = XGBRegressor( n_estimators=100,eval_metric=['rmse'] )

xgbr.fit(x_train,y_train,eval_set=[[x_train,y_train],[x_test,y_test]])

y_preds_xgb = xgbr.predict(x_test)
   
print(np.sqrt(mean_squared_error(y_test, y_preds_xgb)))

[0]	validation_0-rmse:40.31726	validation_1-rmse:40.24028
[1]	validation_0-rmse:33.15944	validation_1-rmse:33.38758
[2]	validation_0-rmse:28.91496	validation_1-rmse:29.43437
[3]	validation_0-rmse:26.20819	validation_1-rmse:27.11546
[4]	validation_0-rmse:24.74820	validation_1-rmse:25.95772
[5]	validation_0-rmse:23.89339	validation_1-rmse:25.32459
[6]	validation_0-rmse:23.24269	validation_1-rmse:24.95307
[7]	validation_0-rmse:22.86537	validation_1-rmse:24.79649
[8]	validation_0-rmse:22.58823	validation_1-rmse:24.67169
[9]	validation_0-rmse:22.27826	validation_1-rmse:24.60619
[10]	validation_0-rmse:22.15188	validation_1-rmse:24.54207
[11]	validation_0-rmse:21.84776	validation_1-rmse:24.39429
[12]	validation_0-rmse:21.68417	validation_1-rmse:24.33920
[13]	validation_0-rmse:21.57251	validation_1-rmse:24.33828
[14]	validation_0-rmse:21.27825	validation_1-rmse:24.22191
[15]	validation_0-rmse:21.10148	validation_1-rmse:24.17142
[16]	validation_0-rmse:20.89634	validation_1-rmse:24.12907
[17]	va

In [21]:
import optuna
def run(trial):
#     fold = 0
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)

    model = XGBRegressor(
        random_state=42,
        n_estimators=15,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(x_train, y_train, early_stopping_rounds=300, eval_set=[(x_test, y_test)],eval_metric=['rmse'], verbose=1000)
    preds_valid = model.predict(x_test)
    check_arr =[]
    for i in range(len(preds_valid)):
        check_arr.append(preds_valid[i])
    score = np.sqrt(mean_squared_error(y_test, check_arr))
    return score

In [22]:
study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=50)

[I 2022-04-15 10:06:30,850] A new study created in memory with name: no-name-0ac42ee9-4556-466a-bcee-b3c7370ed95f


[0]	validation_0-rmse:49.85866
[14]	validation_0-rmse:34.87196


[I 2022-04-15 10:06:31,059] Trial 0 finished with value: 34.871957594645814 and parameters: {'learning_rate': 0.04424776011932231, 'reg_lambda': 36.37739636254927, 'reg_alpha': 5.1234782879406105e-06, 'subsample': 0.6371207893775518, 'colsample_bytree': 0.8710592306801788, 'max_depth': 6}. Best is trial 0 with value: 34.871957594645814.


[0]	validation_0-rmse:46.59545
[14]	validation_0-rmse:25.78359


[I 2022-04-15 10:06:31,272] Trial 1 finished with value: 25.783595387921757 and parameters: {'learning_rate': 0.12644485542559586, 'reg_lambda': 2.971393556426178, 'reg_alpha': 0.00508161274542331, 'subsample': 0.4734951495584243, 'colsample_bytree': 0.9867369914471881, 'max_depth': 5}. Best is trial 1 with value: 25.783595387921757.


...

[I 2022-04-15 10:06:40,795] Trial 47 finished with value: 24.691185834939308 and parameters: {'learning_rate': 0.219285438517692, 'reg_lambda': 1.054330407316783e-08, 'reg_alpha': 0.001294809937656703, 'subsample': 0.8039678713003809, 'colsample_bytree': 0.5511071982906547, 'max_depth': 6}. Best is trial 30 with value: 24.513070956100147.


[0]	validation_0-rmse:45.74896
[14]	validation_0-rmse:25.59057


[I 2022-04-15 10:06:41,001] Trial 48 finished with value: 25.59056808378045 and parameters: {'learning_rate': 0.1501274698199272, 'reg_lambda': 3.4608533322727e-05, 'reg_alpha': 0.0890777933880371, 'subsample': 0.7602780123566124, 'colsample_bytree': 0.7169050806316526, 'max_depth': 4}. Best is trial 30 with value: 24.513070956100147.


[0]	validation_0-rmse:49.23277
[14]	validation_0-rmse:31.08680


[I 2022-04-15 10:06:41,263] Trial 49 finished with value: 31.086802850914598 and parameters: {'learning_rate': 0.056959833713670686, 'reg_lambda': 3.4875189045151096e-06, 'reg_alpha': 1.3442785516539628, 'subsample': 0.6789190981863107, 'colsample_bytree': 0.6508149826750813, 'max_depth': 7}. Best is trial 30 with value: 24.513070956100147.


In [23]:
study.best_params

{'learning_rate': 0.2039936354326796,
 'reg_lambda': 0.000460703973063578,
 'reg_alpha': 0.20647109921883733,
 'subsample': 0.8091045694531913,
 'colsample_bytree': 0.619826824443532,
 'max_depth': 6}

In [24]:
from catboost import CatBoostRegressor

cbr = CatBoostRegressor(n_estimators=250, eval_metric='RMSE')

cbr.fit(x_train,y_train,eval_set=(x_test,y_test))

y_test_preds_cbc = cbr.predict(x_test)
    
print(np.sqrt(mean_squared_error(y_test, y_test_preds_cbc)))

Learning rate set to 0.16101
0:	learn: 27.3919873	test: 27.6866797	best: 27.6866797 (0)	total: 52ms	remaining: 12.9s
1:	learn: 27.0404370	test: 27.3767043	best: 27.3767043 (1)	total: 54.3ms	remaining: 6.73s
2:	learn: 26.7292691	test: 27.0901269	best: 27.0901269 (2)	total: 56.6ms	remaining: 4.66s
3:	learn: 26.3668620	test: 26.7784565	best: 26.7784565 (3)	total: 58.8ms	remaining: 3.62s
4:	learn: 26.0400994	test: 26.5127516	best: 26.5127516 (4)	total: 61ms	remaining: 2.99s
5:	learn: 25.8445833	test: 26.3903362	best: 26.3903362 (5)	total: 63ms	remaining: 2.56s
6:	learn: 25.6832573	test: 26.2799262	best: 26.2799262 (6)	total: 65.1ms	remaining: 2.26s
7:	learn: 25.5114486	test: 26.1748245	best: 26.1748245 (7)	total: 67.3ms	remaining: 2.04s
8:	learn: 25.3852223	test: 26.0966062	best: 26.0966062 (8)	total: 69.5ms	remaining: 1.86s
9:	learn: 25.2898321	test: 26.0158605	best: 26.0158605 (9)	total: 71.7ms	remaining: 1.72s
10:	learn: 25.1344099	test: 25.9089772	best: 25.9089772 (10)	total: 73.8ms	re

In [25]:
import lightgbm as lgb
lgr = lgb.LGBMRegressor()

lgr.fit(x_train,y_train,eval_set=[[x_train,y_train],[x_test,y_test]])

y_test_preds_lgr = lgr.predict(x_test)

print(np.sqrt(mean_squared_error(y_test, y_test_preds_lgr)))

[1]	training's l2: 751.438	valid_1's l2: 770.057
[2]	training's l2: 720.637	valid_1's l2: 742.282
[3]	training's l2: 695.135	valid_1's l2: 720.015
[4]	training's l2: 673.955	valid_1's l2: 701.467
[5]	training's l2: 656.294	valid_1's l2: 686.845
[6]	training's l2: 640.424	valid_1's l2: 672.918
[7]	training's l2: 627.434	valid_1's l2: 662.666
[8]	training's l2: 615.073	valid_1's l2: 653.954
[9]	training's l2: 603.523	valid_1's l2: 644.841
[10]	training's l2: 595.242	valid_1's l2: 639.312
[11]	training's l2: 587.483	valid_1's l2: 633.472
[12]	training's l2: 580.244	valid_1's l2: 630.138
[13]	training's l2: 573.679	valid_1's l2: 625.267
[14]	training's l2: 567.566	valid_1's l2: 621.134
[15]	training's l2: 560.326	valid_1's l2: 616.585
[16]	training's l2: 554.297	valid_1's l2: 614.289
[17]	training's l2: 549.147	valid_1's l2: 612.266
[18]	training's l2: 544.029	valid_1's l2: 611.018
[19]	training's l2: 538.415	valid_1's l2: 606.994
[20]	training's l2: 534.293	valid_1's l2: 604.753
[21]	trai

In [26]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(x_train,y_train)

y_test_preds_rfr = rfr.predict(x_test)

print(np.sqrt(mean_squared_error(y_test, y_test_preds_rfr)))

23.40474642600591


In [27]:
import optuna
def run(trial):
#     fold = 0
    n_estimators = trial.suggest_int('n_estimators', 2, 200)
    max_depth = int(trial.suggest_loguniform('max_depth', 1, 32))
   
    
    
    model = RandomForestRegressor(
        random_state=42,
        n_estimators=15,
        max_depth=max_depth,
    )
    model.fit(x_train, y_train)
    preds_valid = model.predict(x_test)
    check_arr =[]
    for i in range(len(preds_valid)):
        check_arr.append(preds_valid[i])
    score = np.sqrt(mean_squared_error(y_test, check_arr))
    return score

In [28]:
study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=50)

[I 2022-04-15 10:06:49,522] A new study created in memory with name: no-name-6a8c5aef-a154-4296-bec4-d9bbb8e63b80
[I 2022-04-15 10:06:49,693] Trial 0 finished with value: 25.687715582124248 and parameters: {'n_estimators': 156, 'max_depth': 4.295436278473413}. Best is trial 0 with value: 25.687715582124248.
[I 2022-04-15 10:06:49,760] Trial 1 finished with value: 27.321823166928127 and parameters: {'n_estimators': 27, 'max_depth': 1.2187934431381895}. Best is trial 0 with value: 25.687715582124248.
[I 2022-04-15 10:06:49,828] Trial 2 finished with value: 27.321823166928127 and parameters: {'n_estimators': 133, 'max_depth': 1.6675479748870883}. Best is trial 0 with value: 25.687715582124248.
[I 2022-04-15 10:06:50,343] Trial 3 finished with value: 24.03772679632908 and parameters: {'n_estimators': 177, 'max_depth': 15.657274420205145}. Best is trial 3 with value: 24.03772679632908.
[I 2022-04-15 10:06:50,514] Trial 4 finished with value: 25.687715582124248 and parameters: {'n_estimators

In [29]:
study.best_params

{'n_estimators': 172, 'max_depth': 24.49082832047445}

In [30]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators = 95, max_depth = 24.811419031265416)
rfr.fit(x_train,y_train)

y_test_preds_rfr = rfr.predict(x_test)

print(np.sqrt(mean_squared_error(y_test, y_test_preds_rfr)))

23.39166572560096


In [31]:
fracs = np.linspace(0,1,num = 21)
frac_lgc=[]
frac_xgbc=[]
frac_cbc=[]
frac_rfr = []
blend_score = []
for i in tqdm(fracs):
    for j in fracs:
        for k in fracs:
            for l in fracs:
                if (i+j+k+l) ==1:
                    frac_lgc.append(str(i)+str('*lgc'))
                    frac_xgbc.append(str(j)+str('*xgbc'))
                    frac_cbc.append(str(k)+str('*cbc'))
                    frac_rfr.append(str(l)+str('*rfr'))
                    blend_score.append(np.sqrt(mean_squared_error(y_test, (l*(np.array(y_test_preds_rfr)) + (k*(np.array(y_test_preds_cbc)) + j*(np.array(y_preds_xgb)) + i*(np.array(y_test_preds_lgr)) )))))
                else:
                    pass
                
blend_df = pd.DataFrame({'frac_lgc':frac_lgc,'frac_xgbc':frac_xgbc,'frac_cbc':frac_cbc,'frac_rfr':frac_rfr , 'blend_score':blend_score})
blend_df = blend_df.sort_values(by=['blend_score'])
blend_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/21 [00:00<?, ?it/s]

,frac_lgc,frac_xgbc,frac_cbc,frac_rfr,blend_score
290,0.05*lgc,0.2*xgbc,0.1*cbc,0.65*rfr,23.243847
305,0.05*lgc,0.25*xgbc,0.05*cbc,0.65*rfr,23.244024
472,0.1*lgc,0.2*xgbc,0.05*cbc,0.65*rfr,23.244315
94,0.0*lgc,0.25*xgbc,0.1*cbc,0.65*rfr,23.245034
485,0.1*lgc,0.25*xgbc,0.05*cbc,0.6000000000000001*rfr,23.245584
...,...,...,...,...,...
213,0.0*lgc,0.9*xgbc,0.1*cbc,0.0*rfr,24.034688
214,0.0*lgc,0.9500000000000001*xgbc,0.0*cbc,0.05*rfr,24.065001
401,0.05*lgc,0.9500000000000001*xgbc,0.0*cbc,0.0*rfr,24.101561
215,0.0*lgc,0.9500000000000001*xgbc,0.05*cbc,0.0*rfr,24.107285


In [32]:
preds = 0.05*lgr.predict(test) + 0.2*xgbr.predict(test) + 0.1*cbr.predict(test) +0.65*rfr.predict(test)


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [33]:
sample_df = pd.read_csv(r"../input/cassandra-udyam-2022/sampleSubmission.csv")
sample_df

,Vendor_Name,Number_of_Days_Until_Payment
0,Aaron Hoffman,100
1,Aaron Murphy,100
2,Aaron Parks,100
3,Aaron Thomas,100
4,Abigail Hernandez,100
...,...,...
1120,Zachary Coleman,100
1121,Zachary Dudley,100
1122,Zachary Hernandez,100
1123,Zachary Simmons,100


In [34]:
sample_df['Number_of_Days_Until_Payment'] = preds
sample_df['Number_of_Days_Until_Payment'] = sample_df['Number_of_Days_Until_Payment'].astype(int)
sample_df

,Vendor_Name,Number_of_Days_Until_Payment
0,Aaron Hoffman,24
1,Aaron Murphy,58
2,Aaron Parks,35
3,Aaron Thomas,39
4,Abigail Hernandez,55
...,...,...
1120,Zachary Coleman,40
1121,Zachary Dudley,31
1122,Zachary Hernandez,23
1123,Zachary Simmons,45


In [35]:
sample_df.to_csv('submission_quantile_blend_all.csv',index=False)